Retrieving data from Axelbib
===========================
At this stage the data are within the django data base Axelbib. There they have been 
cleaned and missing information has been manually filled in. 

Axelbib has a django rest framework. 
The data will be retrieved by the rest framework in a json format
and will be push into a pandas dataframe following the Arcas structure.

In [2]:
import requests
import pandas as pd
from itertools import product
import collections
import json
from pandas.io.json import json_normalize

In [3]:
from password import password

In [3]:
keys = ['title', 'provenance', 'score', 'read', 'unique_key',
        'date', 'abstract', 'author', 'journal', 'labels',
        'key_word', 'list_strategies', 'pages', 'key']

In [4]:
def raw_to_db(an_article):
    an_article['author'] = [f['name'] for f in an_article['author']]
    an_article['key_word'] = [f['key_word'] for f in an_article['key_word']]
    an_article['date'] = an_article['date']['year']
    an_article['labels'] = [f['label'] for f in an_article['labels']]
    
    for _, field in enumerate(an_article):
        if not an_article[field]:
            an_article[field] = ['None']
   
    values = []
    for key in keys:
        if type(an_article[key]) is not list:
            values.append([an_article[key]])
        else:
            values.append(an_article[key])
    data = []
    for row in product(*values):
        data.append(row)
    df = pd.DataFrame(data, columns=keys)    
    return df

In [10]:
page = 1
status = 200
dfs = []
while page <= 111:
    req = requests.get('http://127.0.0.1:8000/article/?page={}'.format(page), auth=("nikoleta", password))
    status =  req.status_code
    page += 1
    results_of_req = req.json()['results']
    for raw_article in results_of_req:
        df = raw_to_db(raw_article)
        dfs.append(df)        

In [11]:
df = pd.concat(dfs, ignore_index=True)

In [12]:
len(dfs), len(df), len(df.groupby("unique_key").count())

(1104, 11078, 1104)

In [13]:
df.to_json('data.json')

In [14]:
df.head(10)

,title,provenance,score,read,unique_key,date,abstract,author,journal,labels,key_word,list_strategies,pages,key
0,Network Structures between Strategies in Itera...,arXiv,None,None,00660e6b2eedbebaf611cafc864ed1e4,2014,We use replicator dynamics to study an iterate...,Myungkyoon Roh,"The Korean Physical Society February 2014, Vol...",None,None,None,None,Kim2014
1,Network Structures between Strategies in Itera...,arXiv,None,None,00660e6b2eedbebaf611cafc864ed1e4,2014,We use replicator dynamics to study an iterate...,Seung-Woo Son,"The Korean Physical Society February 2014, Vol...",None,None,None,None,Kim2014
2,Network Structures between Strategies in Itera...,arXiv,None,None,00660e6b2eedbebaf611cafc864ed1e4,2014,We use replicator dynamics to study an iterate...,Young Jin Kim,"The Korean Physical Society February 2014, Vol...",None,None,None,None,Kim2014
3,Effect of Initial Fraction of Cooperators on C...,PLOS,3.2582638,None,0091a50bfd4c106e273c9132da6d33ac,2013,We investigate the influence of initial fracti...,Eriko Fukuda,PLoS ONE,None,None,None,None,Shigaki2013
4,Effect of Initial Fraction of Cooperators on C...,PLOS,3.2582638,None,0091a50bfd4c106e273c9132da6d33ac,2013,We investigate the influence of initial fracti...,Jun Tanimoto,PLoS ONE,None,None,None,None,Shigaki2013
5,Effect of Initial Fraction of Cooperators on C...,PLOS,3.2582638,None,0091a50bfd4c106e273c9132da6d33ac,2013,We investigate the influence of initial fracti...,Keizo Shigaki,PLoS ONE,None,None,None,None,Shigaki2013
6,Effect of Initial Fraction of Cooperators on C...,PLOS,3.2582638,None,0091a50bfd4c106e273c9132da6d33ac,2013,We investigate the influence of initial fracti...,Zhen Wang,PLoS ONE,None,None,None,None,Shigaki2013
7,"The Good, the Bad, and the Rare: Memory for Pa...",PLOS,1.1111743,None,00b1b64eda8418c0e3706f43e894a206,2011,For cooperation to evolve via direct reciproci...,Jeffrey R. Stevens,PLoS ONE,None,None,None,None,Volstorf2011
8,"The Good, the Bad, and the Rare: Memory for Pa...",PLOS,1.1111743,None,00b1b64eda8418c0e3706f43e894a206,2011,For cooperation to evolve via direct reciproci...,Jenny Volstorf,PLoS ONE,None,None,None,None,Volstorf2011
9,"The Good, the Bad, and the Rare: Memory for Pa...",PLOS,1.1111743,None,00b1b64eda8418c0e3706f43e894a206,2011,For cooperation to evolve via direct reciproci...,Jörg Rieskamp,PLoS ONE,None,None,None,None,Volstorf2011
